In [1]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession

# Start SparkSession
spark = SparkSession.builder.appName("CSV Load Example").master("local[*]").getOrCreate()

# Load CSV file into DataFrame
df = spark.read.csv("C:/Users/andre/OneDrive/Dokument/vscode/Examens/aapl_10min_2yr.csv", header=True, inferSchema=True)
# Show the first few rows
df.show()

# Print the schema (column types)
df.printSchema()

+-------------------+-------+-------+-------+-------+-------+
|               date|   open|   high|    low|  close| volume|
+-------------------+-------+-------+-------+-------+-------+
|2023-04-24 15:30:00|164.975| 165.24|164.415| 165.06|60755.0|
|2023-04-24 15:40:00| 165.04| 165.46| 164.94| 165.13|14699.0|
|2023-04-24 15:50:00| 165.13|165.415|165.075| 165.17| 6410.0|
|2023-04-24 16:00:00|165.175| 165.18| 164.82| 164.97|11225.0|
|2023-04-24 16:10:00|164.955| 165.28| 164.82| 165.26|15571.0|
|2023-04-24 16:20:00| 165.26|165.425|165.165|165.295|10654.0|
|2023-04-24 16:30:00|165.285| 165.36| 164.47|  164.5| 8380.0|
|2023-04-24 16:40:00|164.495| 164.52|164.165| 164.25|12247.0|
|2023-04-24 16:50:00| 164.25|  164.4| 164.02|164.035|13455.0|
|2023-04-24 17:00:00| 164.04| 164.23|  163.9| 164.15| 8376.0|
|2023-04-24 17:10:00|164.145| 164.38| 164.07|164.355| 8748.0|
|2023-04-24 17:20:00| 164.36| 164.41| 164.15| 164.17| 6981.0|
|2023-04-24 17:30:00| 164.17| 164.33|  164.1|164.135|12273.0|
|2023-04

In [3]:
spark.stop()

checking a model ignore

In [ ]:
class SASRecBERT(nn.Module):
    def __init__(self, embedding_dim, max_len, hidden_size=128, num_heads=2, dropout_rate=0.1, num_layers=2):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size

        self.embedding_projection = nn.Linear(embedding_dim, hidden_size)
        self.pos_emb = nn.Embedding(max_len, hidden_size)

        self.attention_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=hidden_size,
                nhead=num_heads,
                dim_feedforward=hidden_size * 4,
                dropout=dropout_rate,
                batch_first=True,
                activation="relu"
            )
            for _ in range(num_layers)
        ])

        self.transformer = nn.TransformerEncoder(nn.Sequential(*self.attention_layers), num_layers=num_layers)

        self.output_projection = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, embedding_dim)
        )

    def _normalize_time_weights(self, time_weights, eps=1e-8):
        """Normalize nonzero time weights to [0.1, 1.0], keeping padding values untouched."""
        weights = time_weights.clone()
        non_zero_mask = weights > 0
        if non_zero_mask.any():
            non_zero_weights = weights[non_zero_mask]
            min_val = non_zero_weights.min()
            max_val = non_zero_weights.max()
            if min_val != max_val:
                weights[non_zero_mask] = (non_zero_weights - min_val) / (max_val - min_val) * 0.9 + 0.1
        weights[weights == 0] = eps  # Prevent zero masking issues
        return weights
    
    def forward(self, sequence, time_weights):
        batch_size, seq_len, _ = sequence.shape
        device = sequence.device

        # Compute padding mask: True for padding positions
        padding_mask = (time_weights == 0).bool()

        # Normalize time weights (ensuring non-zero values)
        norm_time_weights = self._normalize_time_weights(time_weights)

        # Project input embeddings
        x = self.embedding_projection(sequence)

        # Add positional encodings
        positions = torch.arange(seq_len, device=device).unsqueeze(0).expand(batch_size, -1)
        pos_emb = self.pos_emb(positions)
        x = x + pos_emb  # [B, L, H]

        # Scale embeddings using normalized time weights
        x = x * norm_time_weights.unsqueeze(-1)

        # Pass through transformer with padding mask
        x = self.transformer(x, src_key_padding_mask=padding_mask)

        # Extract last non-padded token
        last_indices = torch.sum(time_weights > 0, dim=1) - 1
        last_indices = torch.clamp(last_indices, min=0)
        batch_indices = torch.arange(batch_size, device=device)
        final = x[batch_indices, last_indices]  # [B, H]

        return self.output_projection(final)